In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=4

In [ ]:
import argparse
import os
import sys
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from six.moves import cPickle
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
class model():
    def __init__(self,profile):
        self.x = tf.placeholder(tf.float32, [None, 784])
        self.y = tf.placeholder(tf.int32,[None,10])
        self.profile = profile

        self.W1 = tf.get_variable(initializer=tf.random_uniform([784, 100],-0.1,0.1),name="W1")
        self.b1 = tf.get_variable(initializer=tf.random_uniform([100],-0.1,0.1),name="b1")
        self.W2 = tf.get_variable(initializer=tf.random_uniform([100, 100],-0.1,0.1),name="W2")
        self.b2 = tf.get_variable(initializer=tf.random_uniform([100],-0.1,0.1),name="b2")
        self.W3 = tf.get_variable(initializer=tf.random_uniform([100, 10],-0.1,0.1),name="W3")
        self.b3 = tf.get_variable(initializer=tf.random_uniform([10],-0.1,0.1),name="b3")
        
        def half_exp(n,k=1,dtype='float32'):
            n_ones = int(n/2)
            n_other = n - int(n/2)
            return np.append(np.ones(n_ones,dtype=dtype),np.exp((1-k)*np.arange(n_other),dtype=dtype))

        if profile == "linear":
            self.r1 = tf.get_variable(initializer=np.linspace(1,0,num=784,endpoint=False,dtype='float32'),name="r1",dtype='float32')
            self.r2 = tf.get_variable(initializer=np.linspace(1,0,num=100,endpoint=False,dtype='float32'),name="r2",dtype='float32')
            self.r3 = tf.get_variable(initializer=np.linspace(1,0,num=100,endpoint=False,dtype='float32') ,name="r3",dtype='float32')
        elif profile == "all-one":
            self.r1 = tf.get_variable(initializer=np.ones(784,dtype='float32'),name="r1",dtype='float32')
            self.r2 = tf.get_variable(initializer=np.ones(100,dtype='float32'),name="r2",dtype='float32')
            self.r3 = tf.get_variable(initializer=np.ones(100,dtype='float32'),name="r3",dtype='float32')
        elif profile == "half-exp":
            self.r1 = tf.get_variable(initializer=half_exp(784,2),name="r1",dtype='float32')
            self.r2 = tf.get_variable(initializer=half_exp(100,2),name="r2",dtype='float32')
            self.r3 = tf.get_variable(initializer=half_exp(100,2),name="r3",dtype='float32')
        else:
            self.r1 = tf.get_variable(initializer=np.array(1.0/(np.arange(784)+1),dtype='float32'),name="r1",dtype='float32')
            self.r2 = tf.get_variable(initializer=np.array(1.0/(np.arange(100)+1),dtype='float32'),name="r2",dtype='float32')
            self.r3 = tf.get_variable(initializer=np.array(1.0/(np.arange(100)+1),dtype='float32') ,name="r3",dtype='float32')
        self.learning_rate = 0.001
        # calculate r1*(W1*x)+b
        #z1 = tf.add(tf.multiply(self.r1,tf.matmul(self.x,self.W1)),self.b1)
        #z1 = tf.add(tf.matmul(tf.multiply(self.r1,self.x),self.W1),self.b1)
        z1 = tf.add(tf.matmul(self.x,self.W1),self.b1)
        y1 = tf.nn.relu(z1)
        z2 = tf.add(tf.matmul(tf.multiply(self.r2,y1),self.W2),self.b2)
        #z2 = tf.add(tf.multiply(self.r2,tf.matmul(y1,self.W2)),self.b2)
        y2 = tf.nn.relu(z2)

        #self.logits = tf.add(tf.multiply(self.r3,tf.matmul(y2,self.W3)),self.b3)
        #self.logits = tf.add(tf.matmul(tf.multiply(self.r3,y2),self.W3),self.b3)
        self.logits = tf.add(tf.matmul(y2,self.W3),self.b3)
        self.probs = tf.nn.softmax(self.logits)
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits,labels=self.y)
        self.loss = tf.reduce_mean(cross_entropy)
        
        correct_prediction = tf.equal(tf.argmax(self.probs,1), tf.argmax(self.y,1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        tvars_trainable = tf.trainable_variables()
        self.gamma_vars = [self.r1,self.r2,self.r3]
        
        for rm in self.gamma_vars:
            tvars_trainable.remove(rm)
            print('%s is not trainable.'% rm)
        self.tvars_trainable = tvars_trainable

        tvars = tf.trainable_variables()
        self.tvars = tvars
        self.opt = tf.train.AdamOptimizer(self.learning_rate)
        self.train_op = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss,var_list=self.tvars_trainable)
        self.train_op_gamma = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss,var_list=self.gamma_vars)
        
    def train(self,sess,config,gamma_trainable=False,reuse=False,verbose=True):
        self.learning_rate = config["learning_rate"]
#         # calculate r1*(W1*x)+b
#         #z1 = tf.add(tf.multiply(self.r1,tf.matmul(self.x,self.W1)),self.b1)
#         #z1 = tf.add(tf.matmul(tf.multiply(self.r1,self.x),self.W1),self.b1)
#         z1 = tf.add(tf.matmul(self.x,self.W1),self.b1)
#         y1 = tf.nn.relu(z1)
#         z2 = tf.add(tf.matmul(tf.multiply(self.r2,y1),self.W2),self.b2)
#         #z2 = tf.add(tf.multiply(self.r2,tf.matmul(y1,self.W2)),self.b2)
#         y2 = tf.nn.relu(z2)

#         #self.logits = tf.add(tf.multiply(self.r3,tf.matmul(y2,self.W3)),self.b3)
#         #self.logits = tf.add(tf.matmul(tf.multiply(self.r3,y2),self.W3),self.b3)
#         self.logits = tf.add(tf.matmul(y2,self.W3),self.b3)
#         self.probs = tf.nn.softmax(self.logits)
#         cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits,labels=self.y)
#         self.loss = tf.reduce_mean(cross_entropy)
        
#         correct_prediction = tf.equal(tf.argmax(self.probs,1), tf.argmax(self.y,1))
#         self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
#         tvars_trainable = tf.trainable_variables()
#         self.gamma_vars = [self.r1,self.r2,self.r3]
        
#         if gamma_trainable:
#             print('Only gamma(s) are trainable.')
#             self.tvars_trainable = self.gamma_vars
#         else:
#             for rm in self.gamma_vars:
#                 tvars_trainable.remove(rm)
#                 print('%s is not trainable.'% rm)
#             self.tvars_trainable = tvars_trainable

#         tvars = tf.trainable_variables()
#         self.tvars = tvars
#         self.opt = tf.train.AdamOptimizer(self.learning_rate)
#         self.train_op = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss,var_list=self.tvars_trainable)
#         self.train_op_gamma = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss,var_list=self.gamma_vars)
        
        if not reuse:
            for v in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
                sess.run(v.initializer)
            print("Initialized.")

        epochs = config['epochs']
        batch_per_epoch = config['batch_per_epoch']
        batch_size = config['batch_size']
        save_dir = config['save_dir']
        mnist = config['mnist']

        #saver = tf.train.Saver(tf.global_variables())
        val_loss_per_epoch = []
        val_accu_per_epoch = []
        loss_per_epoch = []
        accu_per_epoch = []
        for epoch in range(0,epochs):
            start_time = time.time()
            train_loss = []
            train_accu = []
            for batch in range(0,batch_per_epoch):
                batch_x, batch_y = mnist.train.next_batch(batch_size)
                if gamma_trainable:
                    _,loss, accu = sess.run([self.train_op_gamma, self.loss, self.accuracy], 
                                            feed_dict = {self.x:batch_x, self.y: batch_y})
                else:
                    _,loss, accu = sess.run([self.train_op, self.loss, self.accuracy], 
                                            feed_dict = {self.x:batch_x, self.y: batch_y})
                train_loss.append(loss)
                train_accu.append(accu)
            loss_per_epoch.append(np.mean(train_loss))
            accu_per_epoch.append(np.mean(train_accu))
            
            # test after one epoch
            val_loss , probs, val_accu = sess.run([self.loss,self.probs,self.accuracy],feed_dict={self.x:mnist.test.images, self.y:mnist.test.labels})
            val_loss_per_epoch.append(val_loss)
            val_accu_per_epoch.append(val_accu)
            if verbose:
                print(" epoch: %3d \t time: %3f \t loss: %3f \t val_loss: %3f \t accu: %3f \t val_accu: %3f"% 
                  (epoch,time.time() - start_time,np.mean(train_loss), val_loss, np.mean(train_accu), val_accu))
        return({'loss':loss_per_epoch, 'val_loss':val_loss_per_epoch,'accu':accu_per_epoch,'val_accu':val_accu_per_epoch})
    
    def predict(self,sess,config,idp,scope):
        with tf.variable_scope(scope):
            # use variable t to control the on/off of neurons
            n_ones = int(784 * idp)
            n_zeros = 784 - n_ones
            t1 = tf.get_variable(initializer=np.append(np.ones(n_ones,dtype='float32'),np.zeros(n_zeros,dtype='float32')),name="t1",dtype='float32')

            n_ones = int(100 * idp)
            n_zeros = 100 - n_ones
            t2 = tf.get_variable(initializer=np.append(np.ones(n_ones,dtype='float32'),np.zeros(n_zeros,dtype='float32')),name="t2",dtype='float32')

            n_ones = int(100 * idp)
            n_zeros = 100 - n_ones
            t3 = tf.get_variable(initializer=np.append(np.ones(n_ones,dtype='float32'),np.zeros(n_zeros,dtype='float32')),name="t3",dtype='float32')

            p1 = tf.get_variable(initializer=tf.multiply(self.r1,t1),name="p1")
            p2 = tf.get_variable(initializer=tf.multiply(self.r2,t2),name="p2")
            p3 = tf.get_variable(initializer=tf.multiply(self.r3,t3),name="p3")

            for v in [t1,t2,t3,p1,p2,p3]:
                sess.run(v.initializer)
            print("Running IDP mechanism {:.0f}%".format(idp*100),end='\r')
            
            #sess.run(tf.pack([tf.is_variable_initialized(v) for v in [t1,t2,t3,p1,p2,p3]]))
            #print("Initialized IDP mechanism.")

            #z1 = tf.add(tf.multiply(self.r1,tf.matmul(self.x,self.W1)),self.b1)
            #z1 = tf.add(tf.matmul(tf.multiply(self.r1,self.x),self.W1),self.b1)
            z1 = tf.add(tf.matmul(self.x,self.W1),self.b1)
            y1 = tf.nn.relu(z1)
            # IDP on layer 2
            #z2 = tf.add(tf.multiply(p2,tf.matmul(y1,self.W2)),self.b2)
            z2 = tf.add(tf.matmul(tf.multiply(p2,y1),self.W2),self.b2)
            y2 = tf.nn.relu(z2)
            #logits = tf.add(tf.multiply(self.r3,tf.matmul(y2,self.W3)),self.b3)
            #logits = tf.add(tf.matmul(tf.multiply(self.r3,y2),self.W3),self.b3)
            logits = tf.add(tf.matmul(y2,self.W3),self.b3)
            self.pred_probs = tf.nn.softmax(logits)

            mnist = config['mnist']
            pred_probs = sess.run([self.pred_probs],
                                  feed_dict={self.x:mnist.test.images, self.y:mnist.test.labels})
            return(pred_probs)

In [ ]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [ ]:
profile_test = ["all-one","linear","harmonic","half-exp"]

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [ ]:
counter = 0

In [ ]:
counter = counter + 1
for profile in profile_test:
    print(profile)
    with tf.variable_scope(profile+str(counter)):
        model_test = model(profile=profile)
        save_dir = "~/IDP/output/1_"
        epochs = 40
        batch_per_epoch = 200
        batch_size = 28
        learning_rate = 0.001
        config = {'epochs': epochs,
                  'batch_per_epoch': batch_per_epoch,
                  'batch_size': batch_size,
                  'learning_rate': learning_rate,
                  'save_dir': save_dir,
                  'log_file': save_dir+profile+"_log.csv",
                  'result_file': save_dir+profile+"_result.csv",
                  'mnist': mnist}
        with tf.Session() as sess:
            log = model_test.train(sess=sess,config=config,gamma_trainable=False,verbose=False)
            log = pd.DataFrame.from_dict(log)
            log.to_csv(config['log_file'],index=None)
            print(log)
            
            true_lab = [np.argmax(ite) for ite in mnist.test.labels]
            profile_arr = []
            idp_arr = []
            accu_arr = []
            for idp in np.arange(0.1,1.05,0.05):  
                probs = model_test.predict(sess=sess,config=config,idp=idp,scope='i'+str(int(idp*100)))
                pred_lab = [np.argmax(ite) for ite in probs[0]]
                accu = accuracy_score(y_pred=pred_lab,y_true=true_lab)
                profile_arr.append(profile)
                idp_arr.append(idp)
                accu_arr.append(accu)
            log = pd.DataFrame.from_dict({'profile':profile_arr,'IDP':idp_arr,'accu':accu_arr})
            log.to_csv(config['result_file'],index=None)
            print(log)

In [ ]:
profile = "harmonic"

In [ ]:
save_dir = "~/IDP/output/1_"
epochs = 100
batch_per_epoch = 200
batch_size = 28
learning_rate = 0.001
config = {'epochs': epochs,
          'batch_per_epoch': batch_per_epoch,
          'batch_size': batch_size,
          'learning_rate': learning_rate,
          'save_dir': save_dir,
          'log_file': save_dir+profile+"_log.csv",
          'result_file': save_dir+profile+"_result.csv",
          'mnist': mnist}

In [ ]:
with tf.Session() as sess:
    with tf.variable_scope("run2") as scope:
        model_test = model(profile)
        log = model_test.train(sess=sess,config=config,gamma_trainable=False,reuse=False,verbose=False)
        log = pd.DataFrame.from_dict(log)
        #log.to_csv(config['log_file'],index=None)
        #print(log)

        true_lab = [np.argmax(ite) for ite in mnist.test.labels]
        profile_arr = []
        idp_arr = []
        accu_arr = []
        for idp in np.arange(0.1,1.05,0.05):  
            probs = model_test.predict(sess=sess,config=config,idp=idp,scope='i'+str(int(idp*100)))
            pred_lab = [np.argmax(ite) for ite in probs[0]]
            accu = accuracy_score(y_pred=pred_lab,y_true=true_lab)
            profile_arr.append(profile)
            idp_arr.append(idp)
            accu_arr.append(accu)
        log = pd.DataFrame.from_dict({'profile':profile_arr,'IDP':idp_arr,'accu':accu_arr})
        #log.to_csv(config['result_file'],index=None)
        print(log)
        
        print("============ FURTHER TRAINING ============")
        scope.reuse_variables()
        
        config['epochs'] = 50
        config['learning']
        log = model_test.train(sess=sess,config=config,gamma_trainable=True,reuse=True,verbose=False)
        log = pd.DataFrame.from_dict(log)
        #log.to_csv(config['log_file'],index=None)
        #print(log)

        true_lab = [np.argmax(ite) for ite in mnist.test.labels]
        profile_arr = []
        idp_arr = []
        accu_arr = []
        for idp in np.arange(0.1,1.05,0.05):  
            probs = model_test.predict(sess=sess,config=config,idp=idp,scope='i'+str(int(idp*100)))
            pred_lab = [np.argmax(ite) for ite in probs[0]]
            accu = accuracy_score(y_pred=pred_lab,y_true=true_lab)
            profile_arr.append(profile)
            idp_arr.append(idp)
            accu_arr.append(accu)
        log = pd.DataFrame.from_dict({'profile':profile_arr,'IDP':idp_arr,'accu':accu_arr})
        #log.to_csv(config['result_file'],index=None)
        print(log)